In [1]:
# Importing necessary libraries
import logging
from tulip import spec
from tulip import synth
from tulip.transys import machines
from tulip import dumpsmach

logging.basicConfig(level=logging.WARNING)

# User inputs:
# File name to store control strategy:
fname = "strategy_simple_runner_blocker.py"
class_name = "runner"

`omega.symbolic.symbolic` failed to import `dd.cudd`.
Will use `dd.autoref`.


In [2]:
# Modeling environment variables:
env_vars = {}
env_vars['Cr'] = (1,5)
env_init = {'(Cr = 2)'} # empty set
# Car is not patrolling. It is moving anywhere in the center two columns
env_safe = {'(Cr = 2) -> X(Cr = 3)', 
            '(Cr = 3) -> X(Cr = 2 || Cr = 4)', 
            '(Cr = 4) -> X(Cr = 3)', 
            '(Cr = 1) -> X(Cr = 1)', 
            '(Cr = 5) -> X(Cr = 5)'
            } 
env_prog = {} # Env. has no goal

In [3]:
# Modeling system variables:
# Goal vertex is modified to be a sink vertex. Then reachability spec eventually(goal) 
# is the same as Buchi spec. always eventually (goal).
sys_vars = {}
sys_vars['Xr'] = (1, 5)
sys_init = {'Xr=1'}
sys_safe = {'(Xr=1) -> X(Xr = 1||Xr = 2||Xr = 3||Xr = 4)',
            '(Xr = 2) -> X(Xr = 1||Xr = 2||Xr = 3||Xr = 5)',
            '(Xr = 3) -> X(Xr = 1||Xr = 2||Xr = 3||Xr = 4||Xr = 5)',
            '(Xr = 4) -> X(Xr = 1||Xr = 3||Xr = 4||Xr = 5)',
            '(Xr = 5) -> X(Xr =5)'} # Sink vertex
sys_prog = {'Xr=5'} # []<>(Xr = 5)

In [4]:
# Creating GR(1) specification:
specs = spec.GRSpec(env_vars, sys_vars, env_init, sys_init, env_safe, sys_safe, env_prog, sys_prog)
specs.qinit = '\E \A'
strategy = synth.synthesize(specs)
assert strategy is not None, 'unrealizable'


removed 0 nodes from 6 total


In [5]:
# Save strategy to file:
dumpsmach.write_python_case(fname, strategy, classname=class_name)